<a href="https://colab.research.google.com/github/ttjh1234/Study_MLDL/blob/main/chapter05_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#트리의 앙상블

chapter05_03에서는 정형 데이터를 다루는데 가장 뛰어난 성과를 내는 **앙상블 학습**을 한다.

데이터는 chapter05에서 계속 쓰고 있는 wine_data를 계속 사용한다.

##랜덤 포레스트

**랜덤 포레스트(Random Forest)** 는 앙상블 학습의 대표 주자 중 하나이다.

결정 트리를 랜덤하게 만들어 결정 트리의 집합을 만든다.

랜덤 포레스트에서는 **부트스트랩 샘플(bootstrap sample)**을 사용한다.

부트스트랩은 data_set에서 **중복을 허용**하여 데이터를 샘플링하는 방식을 의미한다.

sklearn의 random forest는 기본적으로 100개의 결정트리를 훈련을 하고, 

**분류**일때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 판단,

**회귀**일때는 단순히 각 트리의 예측을 평균한다.

또한 결정트리를 만들 시, 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 랜덤하게 고른 다음, 

이 중에서 최선의 분할을 찾는다. (**기본적으로 전체 특성 개수의 제곱근만큼 특성 선택**)

다만, **RandomForestRegressor는 전체 특성을 사용한다.**

**sklearn.ensemble**의 **RandomForestClassifier** 를 사용하여 랜덤포레스트를 구현할 수 있다.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine_data=pd.read_csv('https://bit.ly/wine-date')
x=wine_data[['alcohol','sugar','pH']].to_numpy()
y=wine_data['class'].to_numpy()
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

데이터를 가져와서, 모형학습을 위해 numpy배열로 만들고, train_set과 test_set으로 나눈다.

그 후 전 장에서 배운 cross_validate()함수를 사용하여 교차 검증을 한다.

RandomForestClassifier는 기본적으로 100개의 트리를 사용하므로, 

n_jobs 매개변수를 -1로 설정하여 모든 CPU 코어를 사용한다.

또한 교차검증에서 **return_train_score** 매개변수를 사용하면, train_set에 대한 점수를 반환한다.




In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1, random_state=42)

In [3]:
scores=cross_validate(rf,x_train,y_train,return_train_score=True,n_jobs=-1)
print(scores['train_score'],'\n',scores['test_score'])

[0.9971133  0.99663219 0.9978355  0.9973545  0.9978355 ] 
 [0.88461538 0.88942308 0.90279115 0.88931665 0.88642926]


In [4]:
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


위의 출력된 결과를 보면 train_set에 다소 과대적합이긴 하지만, 매개변수를 조정하진 않겠다.

랜덤포레스트도 결정 트리의 앙상블이기 때문에, DecisionTreeClassifier가 제공하는 매개변수를 제공한다.

**(criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf)**

결정트리의 큰 장점 중 하나인 특성 중요도를 계산할 수 있다.

In [5]:
rf.fit(x_train,y_train)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


랜덤포레스트에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다.

부트스트랩 샘플을 만들어 훈련을 하고 나면, 부트스트랩 샘플에 포함되지 않은 데이터들이 있다. 

이를 **OOB샘플(Out of Bag Sample)**이라고 한다.

이 샘플을 사용하여 모델을 평가하려면, RandomForesetClassifier의 **oob_score** 매개변수를 사용한다.


In [7]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(x_train,y_train)
print(rf.oob_score_)

0.8934000384837406


##엑스트라 트리

**엑스트라 트리(Extra Tree)** 는 랜덤 포레스트와 비슷하게 동작한다.

기본적으로 100개의 트리를 훈련하고, 결정트리의 여러가지 매개변수를 사용할 수 있다.

전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드의 분할에 사용.

하지만, 부트스트랩 샘플을 사용하지 않고, 각 결정 트리를 만들때, 전체 train_set을 사용한다.

대신 노드를 분할을 할 때, 무작위로 분할을 한다.

(DecisionTreeClassifer의 splitter 매개변수를 random으로 사용한 트리)

보통 엑스트라 트리가 무작위성이 더 크기 때문에, 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 하지만, 랜덤하게 노드를 분할해서 계산 속도가 빠르다.

**sklearn.ensemble**의 **ExtraTreesClassifier**를 통해 Extra Tree를 사용할 수 있다.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [10]:
et.fit(x_train,y_train)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


##그레이디언트 부스팅

**그레이디언트부스팅(Gradient Boosting)**은 깊이가 얇은 결정 트리를 사용하여 이진 트리의 오차를 보완하는 방식으로 앙상블을 한다.

sklearn의 GradientBoostingClassifier는 기본적으로 깊이가 3인 결정트리를 100개 사용한다.

깊이가 얕은 결정트리를 사용하기에, 과대적합에 강하고, 

일반적으로 높은 일반화 성능을 기대할 수 있다.

경사 하강법을 사용하여 트리를 앙상블에 추가한다.

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.

학습률을 증가시키고, 트리의 개수를 늘리면 성능이 향상될 수 있다.

learning_rate의 기본값은 0.1이다.

In [12]:
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores=cross_validate(gb,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(x_train,y_train)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


트리 훈련에 사용할 train_set의 비율을 정하는 **subsample** 매개변수의 기본값은 1.0으로

전체 train_set을 사용한다.

하지만 subsample이 1보다 작으면 train_set의 일부를 사용하며,

이는 확률적 경사 하강법이나 미니비치 경사 하강법과 비슷하다.

일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만,

순서대로 트리를 추가하기 때문에 훈련속도가 느리다.

그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것이 **히스토그램 기반 그레이디언트 부스팅**이다.

##히스토그램 기반 그레이디언트 부스팅

**히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)**은 먼저 

입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때, 최적의 분할을 빠르게 찾을 수 있다.

**히스토그램 기반 그레이디언트부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을**

**위해 사용하여, 입력에 누락된 특성이 있더라도 전처리할 필요가 없다.**

일반적으로 HistGradientBoostingClassifier는 기본 매개변수에서도 안정적인 성능을 낸다.

HistGradientBoostingClassifier는 트리의 개수를 지정하는데, **max_iter** 매개변수를 사용

sklearn의 히스토그램 기반 그레이디언트 부스팅은 **테스트 과정**에 있어,

**sklearn.experimental** 패키지 아래에있는 **enable_hist_gradient_boosting** 모듈을 임포트해야한다.

In [14]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 

**permutation_importace()** 함수를 사용한다. 

이 함수는 특성을 하나씩 랜덤하게 섞어 모델의 성능이 변화하는지 관찰하는 식으로 

어떤 특성이 중요한지 계산한다. 

또한 훈련 세트 뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용할 수 있다.

먼저 히스토그램 기반 그레이디언트 부스팅 모델을 훈련하고 훈련 세트에서 특성 중요도를 계산해 보겠습니다. n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정합니다. 여기서는 10으로 지정하겠습니다. 기본값은 5입니다.

In [16]:
from sklearn.inspection import permutation_importance
hgb.fit(x_train,y_train)
result=permutation_importance(hgb,x_train,y_train,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 반복하여 얻은 

특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)를 담고 있다.

평균을 출력해 보면 랜덤 포레스트와 비슷한 비율임을 알 수 있다. 

이번에는 test_set에서 특성 중요도를 계산해 보겠습니다.


In [18]:
result=permutation_importance(hgb,x_test,y_test,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 

조금 더 당도에 집중하고 있다는 것을 알 수 있다. 

이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있다.

In [19]:
hgb.score(x_test,y_test)

0.8723076923076923

##XGBoost와 LightGBM

sklearn에서 제공하는 히스토그램 기반 그레이디언트 부스팅은 비교적 새로운 기능이다.

sklearn말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리가 있다.

대표적으로 XGBoost, LightGBM 이다.

XGBoost는 sklearn의 cross_validate와도 함께 사용할 수 있고,

다양한 부스팅 알고리즘을 지원한다. tree_method 매개변수에 hist로 지정하면

히스토그램 기반 그레이디언트 부스팅을 사용할 수 있다.

LightGBM은 마이크로소프트에서 만든 라이브러리이다.

sklearn의 HistGradientBoosting이 LightGBM에서 영향을 많이 받았다.

In [20]:
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist',random_state=42)
scores=cross_validate(xgb,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [21]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores=cross_validate(lgb,x_train,y_train,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
